In [ ]:
import os

In [ ]:
# Check present working directory
%pwd

In [ ]:
# Change present working directory to thr root directory
os.chdir('../')

In [ ]:
%pwd

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    untar_path: Path

In [ ]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories
from pathlib import Path

In [ ]:
class ConfigurationManager:
    CONFIG_FILE_PATH = r'C:\Users\deept\ShopTalk\config\config.yaml'
    PARAMS_FILE_PATH = r"C:\Users\deept\ShopTalk\params.yaml"
    SCHEMA_FILE_PATH = r"C:\Users\deept\ShopTalk\schema.yaml"
    def __init__(
        self,
        config_filepath = Path(CONFIG_FILE_PATH),
        params_filepath = Path(PARAMS_FILE_PATH),
        schema_filepath = Path(SCHEMA_FILE_PATH)):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir, 
            untar_path=config.untar_path 
        )

        return data_ingestion_config

In [ ]:
import os
import urllib.request as request
import zipfile
import tarfile
import gzip
from mlProject import logger
from mlProject.utils.common import get_size


In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    
    
    def extract_gz_files(self):
        """
        Extracts .gz files located in the 'metadata' subfolder within the 'listings' directory.
        """
        # Path to the 'metadata' subfolder inside the 'listings' directory
        gz_files_dir = Path(self.config.unzip_dir) / 'listings' / 'metadata'
        os.makedirs(gz_files_dir, exist_ok=True)  # Ensure the 'metadata' directory exists

        # Iterate over .gz files in the 'metadata' directory
        for gz_file in gz_files_dir.glob('*.gz'):
            # Construct the full path to the tar.gz file
            tar_file_path = gz_files_dir / gz_file
            # Open the tar.gz file for extraction
            with gzip.open(tar_file_path, 'rb') as f_in:
                # Construct the file path for the extracted file
                # This assumes the file has a '.json' extension once decompressed
                extracted_file_path = tar_file_path.with_suffix('.json') if tar_file_path.suffixes[-2] == '.tar' else tar_file_path.with_suffix('')
                # Write out the decompressed contents to the new file
                with open(extracted_file_path, 'wb') as f_out:
                    f_out.write(f_in.read())

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_gz_files()
except Exception as e:
    raise e